In [1]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

In [ ]:
(df[pd.isna(df["region_1"]) & pd.isna(df["region_2"])])

In [ ]:
df.info()

In [ ]:
df

In [ ]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(df["designation"].value_counts())

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df["designation"].value_counts()[:100])

In [ ]:
#df[df["designation_tr"] == "reserva" | df["designation_tr"] == "riserva" | df["designation_tr"] == "rèserve"]["designation_tr"] = "reserve"
df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])


In [ ]:
df

In [ ]:
# rimuovo regioni doppie e creo regione unica
df.loc[df["region_1"] == df["region_2"], "region_2"] = ""
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)

In [ ]:
singular_region_complete = set(df["region_complete"])
len(singular_region_complete)

In [ ]:
import geocoder

g = geocoder.arcgis('Redlands, CA')
print(g.latlng)

In [ ]:
import geocoder

singular_region_complete_with_latlong = []
for region in singular_region_complete:
    print(region)
    loc =  geocoder.arcgis(region,maxRows=1)
    print(loc)
    try:
        singular_region_complete_with_latlong.append((region, loc.latlng[0],loc.latlng[1]))   
    except:
        pass

In [ ]:
with open('region_coordinates.csv', 'w') as fp:
    fp.write('\n'.join('%s,%f,%f' % x for x in singular_region_complete_with_latlong))

In [ ]:
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, how="left", on="region_complete")

In [ ]:
df.to_csv("competition_dataset/mine_dev.csv",index=False)

In [ ]:
df = pd.get_dummies(df, columns=["designation_tr","province","variety"])

In [ ]:
df.columns

In [10]:
df = pd.read_csv("competition_dataset/mine_dev.csv")

In [11]:
df

,country,description,designation,province,region_1,region_2,variety,winery,quality,designation_tr,region_complete,lat,long
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0,brut blanc de blancs,Crémant d'Alsace,33.97902,-118.414180
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0,NaN,Paso Robles Central Coast,35.62675,-120.692201
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,NaN,Gewürztraminer,Château Bianca,35.0,NaN,Willamette Valley,45.50012,-122.501480
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0,NaN,NaN,NaN,NaN
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0,natì,Pompeiano,45.43135,9.988320
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120739,Italy,This is a lovely Sauvignon with very character...,NaN,Northeastern Italy,Friuli,NaN,Sauvignon,Plozner,40.0,NaN,Friuli,46.07528,13.079720
120740,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,Northern Spain,Rioja,NaN,Tempranillo Blend,Ramón Bilbao,41.0,reserve,Rioja,-29.41336,-66.855900
120741,Australia,Surprisingly complete and harmonious for a Rie...,NaN,South Australia,Adelaide Hills,NaN,Riesling,Wagtail,26.0,NaN,Adelaide Hills,-34.91118,138.707350
120742,Germany,Broad aromas of ripe yellow fruit and a hint o...,Spätlese,Mosel,NaN,NaN,Riesling,Pamona Cellars,26.0,spätlese,NaN,NaN,NaN


import gmaps
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html

locations = df[['lat', 'long']]
weights = df['quality']
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap_layer)
embed_minimal_html('export.html', views=[fig])

In [ ]:
df["variety"].value_counts()[:50]

In [ ]:
df["province"].value_counts()[:50]

df["region_complete"] = df["region_complete"].replace(r'^\s*$', np.NaN, regex=True)


In [12]:
df = pd.get_dummies(df, columns=["region_complete","variety"])

In [13]:
df.to_csv("competition_dataset/mine_dev_with_dummies.csv",index=False)

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("competition_dataset/mine_dev_with_dummies.csv")

In [4]:
len(df.columns)

1824

In [5]:
from platform import python_version

print(python_version())


3.7.6


In [112]:
# Preprocessing su x_eval

df_eval = pd.read_csv("competition_dataset/eval.tsv", sep="\t")


df_eval["designation_tr"] = df_eval["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df_eval["designation_tr"] = df_eval["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])

df_eval.loc[df_eval["region_1"] == df_eval["region_2"], "region_2"] = ""
df_eval["region_complete"] = df_eval[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df_eval["region_complete"] = df_eval["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

df_eval["region_complete"].fillna('None', inplace=True)
df_eval["variety"].fillna('None', inplace=True)

In [2]:
# Preprocessing su x
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])

df.loc[df["region_1"] == df["region_2"], "region_2"] = ""
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df["region_complete"] = df["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

df["region_complete"].fillna('None', inplace=True)
df["variety"].fillna('None', inplace=True)

In [3]:
#Aggiungo coordinate lat long a x
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, how="left", on="region_complete")

print(df.columns)
lat_mean = df['lat'].mean()
long_mean = df['long'].mean()

df["lat"].fillna(lat_mean, inplace=True)
df["long"].fillna(long_mean, inplace=True)


Index(['country', 'description', 'designation', 'province', 'region_1',
       'region_2', 'variety', 'winery', 'quality', 'designation_tr',
       'region_complete', 'lat', 'long'],
      dtype='object')


In [115]:
#Aggiungo coordinate lat long a x_eval
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df_eval = pd.merge(df_eval, df_region_coordinates, how="left", on="region_complete")

In [4]:
# Aggiungo a x bit di encoding di region_complete e variety
from sklearn.preprocessing import OneHotEncoder

ohc_r = OneHotEncoder(handle_unknown="ignore")
ohc_v = OneHotEncoder(handle_unknown="ignore")

ohRegion = ohc_r.fit_transform(df["region_complete"].values.reshape(-1,1)).toarray()
ohVariety = ohc_v.fit_transform(df["variety"].values.reshape(-1,1)).toarray()

dfOneHot_r = pd.DataFrame(ohRegion, columns=["region_complete_" + str(ohc_r.categories_[0][i]) for i in range(len(ohc_r.categories_[0]))])
dfOneHot_v = pd.DataFrame(ohVariety, columns=["variety_" + str(ohc_v.categories_[0][i]) for i in range(len(ohc_v.categories_[0]))])

df = pd.concat([df,dfOneHot_r,dfOneHot_v], axis=1)

In [107]:
# Aggiungo a x_eval bit di encoding di region_complete e variety

ohRegion = ohc_r.transform(df_eval["region_complete"].values.reshape(-1,1)).toarray()
ohVariety = ohc_v.transform(df_eval["variety"].values.reshape(-1,1)).toarray()

dfOneHot_r = pd.DataFrame(ohRegion, columns=["region_complete_" + str(ohc_r.categories_[0][i]) for i in range(len(ohc_r.categories_[0]))])
dfOneHot_v = pd.DataFrame(ohVariety, columns=["variety_" + str(ohc_v.categories_[0][i]) for i in range(len(ohc_v.categories_[0]))])

df_eval = pd.concat([df_eval,dfOneHot_r,dfOneHot_v], axis=1)

print(dfh.shape, dfh_eval.shape)

(120744, 1825) (30186, 1824)


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english")

wpm = vectorizer.fit_transform(df["description"].fillna(""))

print(wpm)

  (0, 3395)	0.23592986190226323
  (0, 16549)	0.2682533077498277
  (0, 9285)	0.3689930715478062
  (0, 7992)	0.25109101688746954
  (0, 741)	0.12615578860565682
  (0, 10727)	0.18882947514290666
  (0, 27676)	0.25790631240339557
  (0, 3668)	0.1838418275698859
  (0, 14620)	0.17988724433311193
  (0, 28814)	0.256916033137239
  (0, 17836)	0.21909495720258693
  (0, 25380)	0.31593136617446227
  (0, 2049)	0.23534551586595734
  (0, 28442)	0.16752227333304606
  (0, 18619)	0.20277553690499267
  (0, 6693)	0.3879944598570688
  (1, 13572)	0.38584097517283755
  (1, 2507)	0.3323984141128139
  (1, 11108)	0.34885753544992903
  (1, 25413)	0.149538291931055
  (1, 10011)	0.22325395807317186
  (1, 10127)	0.09912163477429979
  (1, 17519)	0.17593852681903827
  (1, 25544)	0.3051778662666655
  (1, 3000)	0.1927361552275735
  :	:
  (120742, 1814)	0.09786739866991497
  (120742, 10709)	0.16201095922339348
  (120742, 10127)	0.06826438208469814
  (120742, 23237)	0.17033704431824728
  (120743, 10534)	0.3364228606115949
  

In [5]:
#Aggiungo a x i bit delle N parole più frequenti nella description 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", binary=True, use_idf=False,norm=False)

wpm = vectorizer.fit_transform(df["description"].fillna(""))

N = 300
freq = sorted(zip(vectorizer.get_feature_names(), wpm.sum(axis=0).tolist()[0]),key=lambda x: x[1], reverse=True)[:N]

words = [ word for word, _ in freq ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
df_words = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df.index)

df = pd.concat([df,df_words], axis=1)

In [14]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, n_jobs=-1,max_features="sqrt")

In [7]:
df.iloc[:,9:]

,designation_tr,region_complete,lat,long,region_complete_Abruzzo,region_complete_Adelaida District Central Coast,region_complete_Adelaide,region_complete_Adelaide Hills,region_complete_Adelaide Plains,region_complete_Aglianico del Beneventano,...,word_wine,word_winery,word_wines,word_wood,word_year,word_years,word_yellow,word_young,word_zest,word_zesty
0,brut blanc de blancs,Crémant d'Alsace,33.979020,-118.414180,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,NaN,Paso Robles Central Coast,35.626750,-120.692201,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,Willamette Valley,45.500120,-122.501480,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,None,29.746793,-52.532050,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,natì,Pompeiano,45.431350,9.988320,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120739,NaN,Friuli,46.075280,13.079720,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120740,reserve,Rioja,-29.413360,-66.855900,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120741,NaN,Adelaide Hills,-34.911180,138.707350,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120742,spätlese,None,29.746793,-52.532050,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df.iloc[:, 11:]
y = df["quality"]
            
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regr.fit(X_train,y_train)
            
y_pred = regr.predict(X_test)

r2_score(y_test, y_pred)

0.6725707524976996

['region_complete_Albana di Romagna  ',
 'region_complete_Albemarle County  ',
 'region_complete_Alcamo  ',
 "region_complete_Aleatico dell'Elba  ",
 'region_complete_Alella  ',
 'region_complete_Alexander Valley Sonoma',
 'region_complete_Alghero  ',
 'region_complete_Alicante  ',
 'region_complete_Alicante-Marina Alta  ',
 'region_complete_Almansa  ',
 'region_complete_Aloxe-Corton  ',
 'region_complete_Alpilles  ',
 'region_complete_Alsace  ',
 'region_complete_Alta Langa  ',
 'region_complete_Alta Mesa Central Valley',
 'region_complete_Alta Valle della Greve  ',
 'region_complete_Alto Adige  ',
 'region_complete_Alto Adige Terlano  ',
 'region_complete_Alto Adige Valle Isarco  ',
 'region_complete_Alto Valle de Uco  ',
 'region_complete_Alto Valle del Río Negro  ',
 'region_complete_Amador County Sierra Foothills',
 'region_complete_Amador-Napa California Other',
 'region_complete_Amarone della Valpolicella  ',
 'region_complete_Amarone della Valpolicella Classico  ',
 'region_com

In [16]:
sorted(zip(list(X.columns[11:]), regr.feature_importances_), key=lambda x: x[1],reverse=True)

[('region_complete_Albana di Romagna  ', 0.043604383913698266),
 ('region_complete_Albemarle County  ', 0.036309217492006954),
 ('variety_Portuguese White', 0.01495953928553097),
 ('region_complete_Capay Valley Central Valley', 0.012608673320987215),
 ('word_tobacco', 0.012393785629763884),
 ('word_scents', 0.009783779059611815),
 ('region_complete_Offida Passerina  ', 0.008579665539862428),
 ('word_bottling', 0.00760083840568501),
 ('region_complete_Chianti  ', 0.007414861854311858),
 ('word_richness', 0.007380921837193747),
 ('word_generous', 0.006958379094288497),
 ('word_stone', 0.006844226012199141),
 ('region_complete_Missouri  ', 0.006798318625393814),
 ('word_estate', 0.006626700677287637),
 ('word_year', 0.0063685193673164225),
 ('word_great', 0.006347308750337856),
 ('region_complete_Calabria  ', 0.005865018798635921),
 ('word_baked', 0.005621860623654008),
 ('word_better', 0.005432592548567095),
 ('word_freshness', 0.005428553050840715),
 ('word_merlot', 0.005410984885089793

In [125]:
X_train["lat"].isnull().sum()


14094

In [92]:
dfh_eval.iloc[:,9:]

,region_complete,region_complete_Abruzzo,region_complete_Adelaida District Central Coast,region_complete_Adelaide,region_complete_Adelaide Hills,region_complete_Adelaide Plains,region_complete_Aglianico del Beneventano,region_complete_Aglianico del Taburno,region_complete_Aglianico del Vulture,region_complete_Agrelo,...,variety_Xynisteri,variety_Yapincak,variety_Zelen,variety_Zibibbo,variety_Zierfandler,variety_Zierfandler-Rotgipfler,variety_Zinfandel,variety_Zlahtina,variety_Zweigelt,variety_Žilavka
0,Santa Cruz Mountains Central Coast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Toro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Saint-Julien,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cahors,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30181,Napa Valley Napa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30182,Dry Creek Valley Sonoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30183,Santa Barbara County Central Coast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30184,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
X_eval = dfh_eval.iloc[:,10:]

y_eval = regr.predict(X_eval)

In [94]:
y_eval

array([55.11174904, 47.33763215, 49.09385491, ..., 48.99112121,
       28.23503571, 38.79995463])

In [95]:
with open("my_submission.csv","w") as f:
    f.write("Id,Predicted\n")
    for i,result in enumerate(y_eval):
        f.write(str(i) + "," + str(y_eval[i])+"\n")